# Model Errors

In this notebook we compare different versions of the Heston MC model, and calculate model errors vs closed form solution.

In [ ]:
from datetime import datetime

import numpy as np
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.timetable import py_to_ts

from src.models.aes import HestonAESMC
from src.models.basic import HestonMCBasic
from src.models.better import HestonMCBetter
from src.models.closed import price_vanilla_call

## Create Dataset
Create the dataset, with MC params, discounts and fwds as required by the [Dataset API](https://qablet-academy.github.io/intro/dataset/)

In [ ]:
# Rate Curve
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

# Forward Curve
ticker = "EQ"
spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))

pricing_datetime = datetime(2023, 12, 31)
dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(pricing_datetime).value,
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {
        "PATHS": 10_000,
        "TIMESTEP": 1 / 1000,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VAR": 0.8,
        "MEANREV": 1.0,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Prices

Define a [Vanilla Option](https://qablet.github.io/qablet-contracts/examples/equity_vanilla/) and calculate price.

In [ ]:
# Create Contract
strike = 100
timetable = Option(
    "USD", ticker, strike=strike, maturity=datetime(2024, 12, 31), is_call=True
).timetable()
print(timetable["events"].to_pandas())

  track                      time op  quantity unit
0       2024-12-31 00:00:00+00:00  >       0.0  USD
1       2024-12-31 00:00:00+00:00  +    -100.0  USD
2       2024-12-31 00:00:00+00:00  +       1.0   EQ


In [ ]:
# Closed form price
ref_price, _ = price_vanilla_call(
    strike,
    1.0,
    ticker,
    dataset,
)
print(f"ref_price: {ref_price:11.6f}")

ref_price:   12.903195


In [ ]:
# Create models, calculate price, and error
models = {
    "Basic": HestonMCBasic(),
    "AES": HestonAESMC(),
    "Better": HestonMCBetter(),  # Uses Log Euler for spot, antithetic paths, and milstein discretization for vol
}

for model_name, model in models.items():
    price, _ = model.price(timetable, dataset)
    print(f"{model_name:20s}: {price:11.6f} {price - ref_price:+11.6f}")

Basic               :   13.005833   +0.102638
AES                 :   12.879431   -0.023764
Better              :   13.005703   +0.102508
